# Orders

✏️ **Exercise**

Today, we will investigate the **orders**, and their associated **review score**.

👉 Our goal is to create a DataFrame with the following features:


| feature_name 	| type 	| description 	|
|:---	|:---:	|:---	|
| `order_id` 	| str 	| the id of the order 	|
| `wait_time` 	| float 	| the number of days between order_purchase_timestamp and order_delivered_customer_date 	|
| `expected_wait_time` 	| float 	| the number of days between order_purchase_timestamp and estimated_delivery_date 	|
| `delay_vs_expected` 	| float 	| if the actual order_delivered_customer_date is later than the estimated delivery date, returns the number of days between the two dates, otherwise return 0 	|
| `order_status` 	| str 	| the status of the order 	|
| `dim_is_five_star` 	| int 	| 1 if the order received a five-star review, 0 otherwise 	|
| `dim_is_one_star` 	| int 	| 1 if the order received a one_star, 0 otherwise 	|
| `review_score` 	| int 	| from 1 to 5 	|
| `number_of_products` 	| int 	| number of products that the order contains 	|
| `number_of_sellers` 	| int 	| number of sellers involved in the order 	|
| `price` 	| float 	| total price of the order paid by customer 	|
| `freight_value` 	| float 	| value of the freight paid by customer 	|
| `distance_customer_seller` 	| float 	| the distance in km between customer and seller (optional) 	|  
  
⚠️ We also want to filter out "non-delivered" orders, unless explicitly specified, otherwise we cannot compute the potential delays.

❓ **Your challenge**: 

- Implement each feature as a separate method within the `Order` class available at `olist/order.py`
- Then, create a method `get_training_data()` that returns the complete DataFrame **without `NaN`s**.

💡 Suggested methodology:
- Use the notebook below to write and test your code step-by-step first
- Then copy the code into `order.py` once you are certain of your code logic
- Focus on the data manipulation logic now, we will analyse the dataset visually in the next challenges

🔥 Notebook best practices (must-read) 👇

<details>
    <summary>▸ <i>click here</i></summary>

From now on, exploratory notebooks are going to become pretty long, and we strongly advise you to follow these notebook principles:
- Code your logic so that your Notebook can always be ran from top to bottom without crashing (Cell --> Run All)
- Name your variables carefully 
- Use dummy names such as `tmp` or `_` for intermediary steps when you know you won't need them for long
- Clear your code and merge cells when relevant to minimize Notebook size (`Shift-M`)
- Hide your cell output if you don't need to see it anymore (double-click on the red `Out[]:` section to the left of your cell).
- Make heavy use of jupyter nbextention `Collapsible Headings` and `Table of Content` (call a TA if you can't find them)
- Use the following shortcuts 
    - `a` to insert a cell above
    - `b` to insert a cell below
    - `dd` to delete a cell
    - `esc` and `arrows` to move between cells
    - `Shift-Enter` to execute cell and move focus to the next one
    - use `Shift + Tab` when you are between method brackets e.g. `groupby()` to get the docs! Repeat a few times to open it permanently

</details>





In [82]:
# Auto reload imported module every time a jupyter cell is executed (handy for olist.order.py updates)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
# Import usual modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [84]:
# Import olist data
from olist.data import Olist
olist = Olist()
data = olist.get_data()

In [85]:
# What datasets do we have access to now ? 
data.keys()

dict_keys(['sellers', 'product_category_name_translation', 'orders', 'order_items', 'customers', 'geolocation', 'order_payments', 'order_reviews', 'products'])

In [86]:
orders = data['orders'].copy() # good practice to be sure not to modify your `data` variable

assert(orders.shape == (99441, 8))

## 1. Code `order.py`

### a) `get_wait_time`
    ❓ Return a Dataframe with:
           order_id, wait_time, expected_wait_time, delay_vs_expected, order_status


🎁 We give you the pseudo-code below 👇 for this first method:

> 1. Inspect the `orders` dataframe
2. Filter the dataframe on `delivered orders`
3. Handle `datetime`
    - Take time to understand what python [`datetime`](https://docs.python.org/3/library/datetime.html) objects are
    - and convert dates from "string" type to "pandas.datetime' using [`pandas.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)
4. Compute `wait_time`
5. Compute `expected_wait_time`
6. Compute `delay_vs_expected`
7. Check the new dataframe 
8. Once you are satisfied with your code, you can carefully copy-paste it from the notebook to to `olist/order.py`

<details>
    <summary>💡Hint</summary>

For both `wait_time` and `delay_vs_expected`, you need to subtract the relevant dates/timestamps to get the time difference between the `pandas.datetime` objects. Then, you can either use [`datetime.timedelta()`](https://docs.python.org/3/library/datetime.html#timedelta-objects) or [`np.timedelta64()`](https://numpy.org/doc/stable/reference/arrays.datetime.html#datetime-and-timedelta-arithmetic) to find out how many days that subtraction represents!

</details>

In [109]:
      def __init__(self):
        # Assign an attribute ".data" to all new instances of Order
        self.data = Olist().get_data()
        self.orders = self.data['orders']
        orders = self.orders.copy()

        # Filter the dataframe on delivered orders
        delivered_orders = orders[orders['order_status'] == 'delivered'].copy()

        # Handle datetime
        delivered_orders['order_purchase_timestamp'] = pd.to_datetime(delivered_orders['order_purchase_timestamp'])
        delivered_orders['order_delivered_customer_date'] = pd.to_datetime(delivered_orders['order_delivered_customer_date'])
        delivered_orders['order_estimated_delivery_date'] = pd.to_datetime(delivered_orders['order_estimated_delivery_date'])

        # Compute wait_time
        delivered_orders['wait_time'] = (delivered_orders['order_delivered_customer_date'] - delivered_orders['order_purchase_timestamp']).dt.days

        # Compute expected_wait_time
        delivered_orders['expected_wait_time'] = (delivered_orders['order_estimated_delivery_date'] - delivered_orders['order_purchase_timestamp']).dt.days

        # Compute delay_vs_expected
        delivered_orders['delay_vs_expected'] = (delivered_orders['order_delivered_customer_date'] - delivered_orders['order_estimated_delivery_date']).dt.days
        delivered_orders['delay_vs_expected'] = delivered_orders['delay_vs_expected'].apply(lambda x: x if x > 0 else 0)

        # Return the relevant columns
        return delivered_orders[['order_id', 'wait_time', 'expected_wait_time', 'delay_vs_expected', 'order_status']]

👀 Check the dataframe you've just created. <br/> 

💪 When your code works, commit it to `olist/order.py` <br/>

🧪 Now, test it by running the following cell 👇 

In [110]:
# Test your code here
from olist.order import Order
Order().get_wait_time()

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,8.0,15,0.0,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13.0,19,0.0,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9.0,26,0.0,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,13.0,26,0.0,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2.0,12,0.0,delivered
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,8.0,18,0.0,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,22.0,23,0.0,delivered
99438,83c1379a015df1e13d02aae0204711ab,24.0,30,0.0,delivered
99439,11c177c8e97725db2631073c19f07b62,17.0,37,0.0,delivered


In [111]:
from nbresult import ChallengeResult
test = Order().get_wait_time()
result = ChallengeResult('wait_time', dve_type=test["delay_vs_expected"].dtype, shape=test.shape, dve_min=test["delay_vs_expected"].min(), dve_max=test["delay_vs_expected"].max())
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/francoisgirard/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/francoisgirard/code/francoisgirard51/04-Decision-Science/02-Statistical-Inference/data-orders/tests
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_wait_time.py::TestWaitTime::test_wait_time PASSED                   [100%]

============================== 1 passed in 0.24s ===============================


💯 You can commit your code:

git add tests/wait_time.pickle

git commit -m 'Completed wait_time step'

git push origin master



### b) `get_review_score`
     ❓ Returns a DataFrame with:
        order_id, dim_is_five_star, dim_is_one_star, review_score

dim_is_$N$_star should contain `1` if review_score=$N$ and `0` otherwise 

<details>
    <summary markdown='span'>Hints</summary>

Think about `Series.map()` or `DataFrame.apply()`
    
</details>

👉 We load the `reviews` for you

In [112]:
reviews = data['order_reviews'].copy()
assert(reviews.shape == (99224,7))
reviews

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
...,...,...,...,...,...,...,...
99219,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5,NaN,NaN,2018-07-07 00:00:00,2018-07-14 17:18:30
99220,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-09 00:00:00,2017-12-11 20:06:42
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43
99222,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,NaN,NaN,2018-07-01 00:00:00,2018-07-02 12:59:13


In [119]:
    def get_review_score(self):
        # Utilisez 'self.reviews' pour accéder aux reviews dans la classe
        self.reviews['dim_is_five_star'] = self.reviews['review_score'].apply(lambda x: 1 if x == 5 else 0)
        self.reviews['dim_is_one_star'] = self.reviews['review_score'].apply(lambda x: 1 if x == 1 else 0)

        # Sélection des colonnes pertinentes
        return self.reviews[['order_id', 'dim_is_five_star', 'dim_is_one_star', 'review_score']]

Once again, 

👀 Check the dataframe you've just created. <br/> 

💪 When your code works, commit it to `olist/order.py` <br/>

🧪 Now, test it by running the following cell 👇 

In [121]:
# Test your code here
from olist.order import Order
Order().get_review_score()

,order_id,dim_is_five_star,dim_is_one_star,review_score
0,73fc7af87114b39712e6da79b0a377eb,0,0,4
1,a548910a1c6147796b98fdf73dbeba33,1,0,5
2,f9e4b658b201a9f2ecdecbb34bed034b,1,0,5
3,658677c97b385a9be170737859d3511b,1,0,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,1,0,5
...,...,...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,1,0,5
99220,22ec9f0669f784db00fa86d035cf8602,1,0,5
99221,55d4004744368f5571d1f590031933e4,1,0,5
99222,7725825d039fc1f0ceb7635e3f7d9206,0,0,4


In [122]:
from nbresult import ChallengeResult
result = ChallengeResult('review_score', shape=Order().get_review_score().shape)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/francoisgirard/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/francoisgirard/code/francoisgirard51/04-Decision-Science/02-Statistical-Inference/data-orders/tests
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_review_score.py::TestReviewScore::test_review_score PASSED          [100%]

============================== 1 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/review_score.pickle

git commit -m 'Completed review_score step'

git push origin master



### c) `get_number_products`:
     ❓ Returns a DataFrame with:
        order_id, number_of_products (total number of products per order)

In [127]:
def get_number_products(self):
    # Accéder aux données des articles commandés
    order_items = self.data['order_items']
    
    # Grouper les données par 'order_id' et compter le nombre d'articles dans chaque commande
    number_of_products_per_order = order_items.groupby('order_id')['order_item_id'].count().reset_index()
    
    # Renommer les colonnes pour correspondre à ce qui est demandé
    number_of_products_per_order = number_of_products_per_order.rename(columns={'order_item_id': 'number_of_products'})
    
    return number_of_products_per_order

🧪 Same routine: 
* check your dataframe, 
* commit your code to `olist/order.py`
* and check that it truly works.

In [128]:
from nbresult import ChallengeResult
result = ChallengeResult('number_products', shape=Order().get_number_products().shape)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/francoisgirard/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/francoisgirard/code/francoisgirard51/04-Decision-Science/02-Statistical-Inference/data-orders/tests
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_number_products.py::TestNumberProducts::test_review_score PASSED    [100%]

============================== 1 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/number_products.pickle

git commit -m 'Completed number_products step'

git push origin master



### d) `get_number_sellers`:
     ❓ Returns a DataFrame with:
        order_id, number_of_sellers (total number of unique sellers per order)
        
<details>
    <summary>▸ <i>Hint</i></summary>

`pd.Series.nunique()`
</details>

In [129]:
def get_number_sellers(self):
    # Accéder aux données des articles commandés
    order_items = self.data['order_items']
    
    # Grouper les données par 'order_id' et compter le nombre de vendeurs uniques dans chaque commande
    number_of_sellers_per_order = order_items.groupby('order_id')['seller_id'].nunique().reset_index()
    
    # Renommer les colonnes pour correspondre à ce qui est demandé
    number_of_sellers_per_order = number_of_sellers_per_order.rename(columns={'seller_id': 'number_of_sellers'})
    
    return number_of_sellers_per_order

In [131]:
from nbresult import ChallengeResult
result = ChallengeResult('number_sellers', shape=Order().get_number_sellers().shape)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/francoisgirard/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/francoisgirard/code/francoisgirard51/04-Decision-Science/02-Statistical-Inference/data-orders/tests
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_number_sellers.py::TestNumberSellers::test_number_seller PASSED     [100%]

============================== 1 passed in 0.08s ===============================


💯 You can commit your code:

git add tests/number_sellers.pickle

git commit -m 'Completed number_sellers step'

git push origin master



### e) `get_price_and_freight`
     Returns a DataFrame with:
        order_id, price, freight_value

<details>
    <summary>▸ <i>Hint</i></summary>

`pd.Series.agg()` allows you to apply one transformation method per column of your groupby object
</details>

In [132]:
def get_price_and_freight(self):
    # Accéder aux données des articles commandés
    order_items = self.data['order_items']
    
    # Grouper les données par 'order_id' et calculer la somme des prix et des valeurs de fret pour chaque commande
    price_and_freight_per_order = order_items.groupby('order_id')[['price', 'freight_value']].sum().reset_index()
    
    return price_and_freight_per_order

In [133]:
from nbresult import ChallengeResult
result = ChallengeResult('price', shape=Order().get_price_and_freight().shape)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/francoisgirard/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/francoisgirard/code/francoisgirard51/04-Decision-Science/02-Statistical-Inference/data-orders/tests
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_price.py::TestPrice::test_price PASSED                              [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/price.pickle

git commit -m 'Completed price step'

git push origin master



### e) [OPTIONAL] `get_distance_seller_customer` 
**(Try  to code this function only after finishing today's challenges - Skip to next section)**

    ❓ Returns a Dataframe with:
        order_id, distance_seller_customer (the distance in km between customer and seller)

💡Have a look at the `haversine_distance` formula we coded for you in the `olist.utils` module

In [146]:
def get_distance_seller_customer(self):
    """
    Returns a DataFrame with:
    order_id, distance_seller_customer
    """
    # Fusionner les ensembles de données pour obtenir les coordonnées du vendeur et du client
    data_merged = self.orders.merge(self.data['customers'], on='customer_id')\
                             .merge(self.data['sellers'], on='seller_id')

    # Calculer la distance haversine pour chaque commande
    data_merged['distance_seller_customer'] = data_merged.apply(
        lambda row: haversine_distance(
            row['seller_lat'],
            row['seller_lng'],
            row['customer_lat'],
            row['customer_lng']
        ),
        axis=1
    )

    # Sélectionner les colonnes pertinentes
    distance_per_order = data_merged[['order_id', 'distance_seller_customer']]
    return distance_per_order

👀 Check your new dataframe and commit your code to olist/order.py when it works. 

In [147]:
def test_columns_exist():
    order = Order()
    df = order.get_distance_seller_customer()
    assert 'order_id' in df.columns, "La colonne 'order_id' est manquante"
    assert 'distance_seller_customer' in df.columns, "La colonne 'distance_seller_customer' est manquante"
    
def test_data_types():
    order = Order()
    df = order.get_distance_seller_customer()
    assert df['order_id'].dtype == object, "Le type de 'order_id' doit être object (string)"
    assert df['distance_seller_customer'].dtype == float, "Le type de 'distance_seller_customer' doit être float"

def test_no_null_values():
    order = Order()
    df = order.get_distance_seller_customer()
    assert df['order_id'].isnull().sum() == 0, "Il y a des valeurs nulles dans 'order_id'"
    assert df['distance_seller_customer'].isnull().sum() == 0, "Il y a des valeurs nulles dans 'distance_seller_customer'"

def test_distance_values():
    order = Order()
    df = order.get_distance_seller_customer()
    assert (df['distance_seller_customer'] >= 0).all(), "Il y a des distances négatives"
    # Vous pouvez également ajouter des tests pour des distances spécifiques si vous avez ces informations

def test_data_consistency():
    order = Order()
    df = order.get_distance_seller_customer()
    unique_orders = df['order_id'].nunique()
    assert unique_orders == len(df), "Il y a des 'order_id' en double"

🧪  Test your code

In [154]:
from nbresult import ChallengeResult

result = ChallengeResult('distance',
    mean = Order().get_distance_seller_customer()['distance_seller_customer'].mean())
result.write()
print(result.check())

KeyError: 'customer_id'

## 2. All at once: `get_training_data`

❓ Time to code `get_training_data` making use of your previous coded methods, to gather all order features in one table

In [152]:
def get_training_data(self, is_delivered=True, with_distance_seller_customer=False):
    """
    Returns a clean DataFrame (without NaN), with the all following columns:
    ['order_id', 'wait_time', 'expected_wait_time', 'delay_vs_expected',
    'order_status', 'dim_is_five_star', 'dim_is_one_star', 'review_score',
    'number_of_products', 'number_of_sellers', 'price', 'freight_value',
    'distance_seller_customer']
    """
    # Use the previously defined methods to get each piece of data
    wait_time_data = self.get_wait_time(is_delivered)
    review_score_data = self.get_review_score()
    number_products_data = self.get_number_products()
    number_sellers_data = self.get_number_sellers()
    price_and_freight_data = self.get_price_and_freight()

    # Merge all the dataframes on 'order_id'
    data = wait_time_data.merge(review_score_data, on='order_id', how='left')
    data = data.merge(number_products_data, on='order_id', how='left')
    data = data.merge(number_sellers_data, on='order_id', how='left')
    data = data.merge(price_and_freight_data, on='order_id', how='left')

    # If requested, also merge the distance_seller_customer data
    if with_distance_seller_customer:
        distance_data = self.get_distance_seller_customer()
        data = data.merge(distance_data, on='order_id', how='left')

    # Drop rows with NaN values
    data = data.dropna()

    return data

🧪  Test it below

In [153]:
from nbresult import ChallengeResult
from olist.order import Order
data = Order().get_training_data()

result = ChallengeResult('training',
    shape=data.shape,
    columns=sorted(list(data.columns))
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/francoisgirard/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/francoisgirard/code/francoisgirard51/04-Decision-Science/02-Statistical-Inference/data-orders/tests
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 2 items

test_training.py::TestTraining::test_training_data_columns PASSED        [ 50%]
test_training.py::TestTraining::test_training_data_shape PASSED          [100%]

============================== 2 passed in 0.04s ===============================


💯 You can commit your code:

git add tests/training.pickle

git commit -m 'Completed training step'

git push origin master



🏁 Congratulations! 

💾 Commit and push your notebook before starting the next challenge.